# Descripción datos disponibles para el proyecto

El propósito de este notebook es documentar y presentar de manera organizada las fuentes de datos disponibles para el análisis.

Alcance temporal (2021-1 a 2025-1)

## 1. Resumen de las fuentes de datos

| Fuente                         | Periodos disponibles | Descripción breve                                           | Fuente de Datos                         |
|---------------------------------|-----------------------|-------------------------------------------------------------|-----------------------------------------|
| Estudiantes                    | 2024-1 a 2025-1        | Datos sociodemográficos y académicos de los estudiantes.     | Excel 2024 y Excel 2025                 |
| Calificaciones                 | 2021-1 a 2025-1        | Notas por asignatura, competencias, estructura de materias. | Excel 2021-2022 (reporte grado-periodo), PDF 2023-2025 (reporte individual) |
| Moodle                         | 2024-1 a 2025-1        | Eventos de interacción académica en plataforma Moodle.      | Base de datos MySQL 2024 y MySQL 2025   |
| Coevaluaciones y Autoevaluaciones | 2024-1 y 2024-2     | Evaluación de investigación por parte de estudiantes.       | Excel por periodo y grado               |
| Docentes                       | 2024-1 a 2025-1        | Datos básicos de docentes: experiencia, edad, nivel educativo. | Excel                                  |
| Carga académica                | 2024-1 a 2025-1        | Intensidad horaria por asignatura y docente.                | Excel                                   |


In [44]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("../../data/datos.csv")

df['año_periodo'] = df['año'].astype(str) + '-' + df['periodo'].astype(str)

df['disponible'] = 1

pivot = df.pivot_table(
    index='fuente',
    columns='año_periodo',
    values='disponible',
    fill_value=0
)

pivot = pivot.reindex(sorted(pivot.columns, key=lambda x: (int(x.split('-')[0]), int(x.split('-')[1]))), axis=1)

fig = px.imshow(
    pivot,
    color_continuous_scale=['white', 'blue'],
    labels=dict(x="Año-Periodo", y="Fuente"),
    title="Mapa de Disponibilidad de Datos por Fuente y Periodo",
)

fig.update_layout(
    template="plotly_white",
    title_font_size=24,
    xaxis_title_font_size=18,
    yaxis_title_font_size=18,
    xaxis_tickangle=-45,
    coloraxis_showscale=False,
    xaxis=dict(type='category')
)
fig.show()

## 2. Estudiantes

### Descripción general
El dataset de estudiantes proporciona información sociodemográfica y académica detallada de los alumnos de la institución educativa. Cubre el periodo de 2024-1 a 2025-1 y fue suministrado en archivos de Excel separados por año.

Esta información puede ser útil para el análisis longitudinal del desempeño académico, participación en Moodle y análisis de factores sociodemográficos relacionados con el aprendizaje.

### Variables disponibles

| Nombre del Campo               | Definición |
|---------------------------------|------------|
| id                              | Identificador único generado por nombre completo + sede, útil cuando falta el documento de identidad. |
| sede                            | Sede educativa a la que pertenece el estudiante (Fusagasugá o Girardot). |
| nivel                           | Nivel académico (Primaria, Bachillerato, etc.) |
| grado                           | Grado escolar actual del estudiante. |
| orden_grado                     | Valor numérico para ordenar los grados académicamente. |
| año_ingreso                     | Año en que el estudiante ingresó a la institución. |
| antigüedad                      | Clasificación como "Nuevo" o "Antiguo" en la institución. |
| género                          | Género del estudiante. |
| documento_identificación        | Número de documento de identidad del estudiante. **(Clave para uniones de datos)** |
| primer_apellido                 | Primer apellido del estudiante. |
| segundo_apellido                | Segundo apellido del estudiante. |
| nombres                         | Nombres del estudiante. |
| fecha_nacimiento                | Fecha de nacimiento del estudiante. |
| demuestra_confianza             | Nivel de confianza percibido en el estudiante. |
| país_origen                     | País de nacimiento del estudiante. |
| estrato                         | Estrato socioeconómico (Colombia). |
| tipo_vivienda                   | Tipo de vivienda: propia, alquilada, familiar, etc. |
| zona_vivienda                   | Zona de residencia (urbana o rural). |
| dirección                       | Dirección residencial del estudiante. |
| horas_semana_estudio_casa       | Estimación de horas de estudio en casa por semana. |
| interés_estudios_superiores     | Nivel de interés en continuar estudios superiores. |
| medio_transporte                | Medio utilizado para llegar a la institución. |
| apoyo_familiar                  | Nivel de apoyo familiar percibido por el estudiante. |
| total_hermanos                  | Número total de hermanos. |
| familia                         | Composición del núcleo familiar. |
| actividades_extracurriculares   | Actividades fuera del horario escolar: deportes, artes, etc. |
| enfermedades                    | Indica si el estudiante presenta alguna enfermedad. |
| proyección_vocacional           | Carrera o área profesional de interés. **(Importante para análisis vocacional)** |
| participación_clase             | Nivel de participación en clase. |
| nee                             | Indica si el estudiante tiene Necesidades Educativas Especiales. |
| valoración_emocional            | Evaluación emocional del estudiante. |
| nivel_motivación                | Nivel general de motivación académica. |

### Variables Clave a destacar:
- **documento_identificación**: Para integrar con otras bases de datos (calificaciones, Moodle, asistencia).
- **sede**: Permite comparaciones entre Fusagasugá y Girardot.
- **antigüedad**: Útil para comparar nuevos y antiguos estudiantes.


## 3. Calificaciones

### Descripción general
El dataset de calificaciones recoge los resultados académicos de los estudiantes desde **2021-1 hasta 2025-1**. La fuente de datos son reportes en Excel (2021–2022) y PDFs individuales (2023–2025), que fueron transformados para su análisis.

Las calificaciones evalúan el desempeño por **asignatura** y por **competencia** en cada periodo académico.

### Cambios importantes en el sistema de evaluación:

- **De 2021 a 2022**:
  - Asignaturas principales: Ciencias Naturales, Ciencias Sociales, Matemáticas, Lengua Castellana, Inglés, Innovación y Emprendimiento, Educación Física, Educación Ambiental, Comprensión Lectora.
  - Competencias evaluadas: **Comunicativa, Procedimental, Axiológica, Actitudinal, Cognitiva** (todas computadas de manera independiente y luego promediadas).

- **De 2023 a 2025**:
  - Cambio en nombres de varias asignaturas.
  - Se **elimina la competencia Comunicativa**.
  - Calificaciones con **promedio ponderado** entre Procedimental, Axiológica, Actitudinal y Cognitiva.

### Tabla de transición de asignaturas (archivo `asignaturas.csv`)

| ID Asignatura | Nombre 2021 | Nombre 2022 | Nombre 2023 | Nombre 2024 | Nombre 2025 | Área |
|---------------|-------------|-------------|-------------|-------------|-------------|------|
| 1             | Ciencias Naturales | Ciencias Naturales | Ciencias Naturales | Ciencias Naturales y Educación Ambiental | Ciencias Naturales y Educación Ambiental | Ciencias Naturales |
| 2             | Ciencias Sociales  | Ciencias Sociales  | Ciencias Sociales  | Ciencias Sociales  | Ciencias Sociales  | Ciencias Sociales |
| 3             | Matemáticas         | Matemáticas         | Matemáticas         | Matemáticas         | Matemáticas         | Matemáticas |
| 4             | Lengua Castellana   | Lengua Castellana   | Lengua Castellana   | Lengua Castellana   | Lengua Castellana   | Humanidades |
| 5             | Inglés              | Inglés              | Inglés              | English             | English             | Humanidades |
| 6             | Innovación y Emprendimiento | Creatividad e Innovación | Creatividad e Innovación | Creatividad e innovación | Investigacion e Innovación |
| 7             |                     | Aprendizaje Basado en Proyectos | Aprendizaje Basado en Proyectos | Aprendizaje Basado en Proyectos | Investigacion e Innovación |
| 8             | Educación Física    | Educación Física    | Educación Física y Deportes | Educación Física y Deportes | Educación Física |
| 9             | Comprensión Lectora | Lectura Crítica     | Lectura Crítica     | Lectura Crítica     | Humanidades |
| ...           | ...                 | ...                 | ...                 | ...                 | ... |

*(Tabla completa disponible en `asignaturas.csv`)*

---

### Variables clave en calificaciones:
- **documento_identificación**: Para unir con estudiantes y demás fuentes.
- **id_asignatura**: Para asociar cada nota al área académica correspondiente.
- **periodo, año**: Para análisis temporal de desempeño.
---

### Notas:
- La transformación de calificaciones de PDFs a CSV implicó la limpieza manual y extracción automática de nombres, documentos y notas individuales.
- Se creó un **hash** para proteger información sensible como documento de identidad durante la etapa de modelado.


In [78]:
import pandas as pd
import plotly.express as px

df_asignaturas = pd.read_csv("../../data/raw/tablas_maestras/asignaturas.csv")

pivot = pd.DataFrame({
    2021: df_asignaturas['nombre_2021'].notna().astype(int),
    2022: df_asignaturas['nombre_2022'].notna().astype(int),
    2023: df_asignaturas['nombre_2023'].notna().astype(int),
    2024: df_asignaturas['nombre_2024'].notna().astype(int),
    2025: df_asignaturas['nombre_2025'].notna().astype(int)
})

index_plus_1 = df_asignaturas.index + 1
mapping = dict(zip(index_plus_1, df_asignaturas['nombre']))

pivot.index = pivot.index + 1   # sumamos +1 al index del pivot
pivot.index = pivot.index.map(mapping)  # mapeamos al nombre_2025

fig = px.imshow(
    pivot,
    labels=dict(x="Año", y="Asignatura (2025)", color="Disponible"),
    title="Disponibilidad de Asignaturas por Año",
    color_continuous_scale=[[0, 'white'], [1, 'blue']]
)

fig.update_layout(
    template="plotly_white",
    width=1200,
    height=800,
    xaxis_title="Año",
    yaxis_title="Asignatura (nombre 2025)",
    coloraxis_showscale=False,
    xaxis=dict(
        type='category',
        tickmode='array',
        tickvals=[2021, 2022, 2023, 2024, 2025],
        ticktext=["2021", "2022", "2023", "2024", "2025"]
    ),
    yaxis=dict(
        tickmode='linear'
    ),
    title_font_size=24
)
fig.show()

In [65]:
pivot

,2021,2022,2023,2024,2025
id_asignatura,,,,,
1,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0
6,0.0,0.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0
9,0.0,0.0,1.0,1.0,1.0


## 4. Moodle

### Descripción general
La fuente de Moodle recoge los eventos de interacción de los estudiantes en la plataforma educativa **Moodle** (versión **2020061503.04**) para los periodos **2024-1 a 2025-1**.

Esta información es fundamental para analizar **autonomía** y **autoaprendizaje** a través del uso de recursos digitales.

### ¿Qué se mide?

- **Interacción en cursos**: Visualización de cursos, páginas, y recursos.
- **Participación activa**: Envío de tareas, publicación en foros, realización de actividades evaluativas como quizzes.
- **Seguimiento de procesos de aprendizaje**: Consultas de retroalimentaciones, revisión de calificaciones, interacción social en la plataforma.

### Asignaturas principales con actividades de aprendizaje en Moodle:

- Ciencias Naturales
- Ciencias Sociales
- Matemáticas
- Lengua Castellana

*(Las demás asignaturas solo cuentan con calendario de actividades publicado.)*

---

### Principales eventos extraídos

| Evento | Descripción |
|:---|:---|
| assign_assessable_submitted | Entrega de tarea o actividad evaluativa. |
| assign_feedback_viewed | El estudiante revisa la retroalimentación de una tarea enviada. |
| assign_module_viewed | Visualiza o abre un módulo de tarea. |
| assign_remove_submission_form_viewed | Consulta la opción para retirar una entrega hecha. |
| assign_submission_form_viewed | Abre el formulario para enviar una tarea. |
| chat_module_viewed | Accede a una sala de chat (comunicación instantánea). |
| choice_answer_created | Selecciona una respuesta en una encuesta o elección. |
| choice_module_viewed | Abre un módulo de encuesta o elección. |
| course_viewed | Ingresa a un curso (actividad general e interés). |
| feedback_module_viewed | Visualiza un módulo de retroalimentación (ej. encuestas). |
| feedback_response_submitted | Envía respuestas a una encuesta o formulario de feedback. |
| file_assessable_uploaded | Sube un archivo que será evaluado (ej: tarea en Word o PDF). |
| file_submission_created | Crea y registra una entrega de archivo. |
| forum_assessable_uploaded | Publica contenido evaluable en un foro. |
| forum_course_searched | Realiza una búsqueda dentro de los foros. |
| forum_discussion_created | Crea una nueva discusión en un foro. |
| forum_discussion_deleted | Elimina una discusión que había creado. |
| forum_discussion_subscription_created | Se suscribe a recibir notificaciones de una discusión específica. |
| forum_discussion_subscription_deleted | Cancela su suscripción a una discusión del foro. |
| forum_discussion_viewed | Lee el contenido de una discusión del foro. |
| forum_module_viewed | Abre el módulo de foro en el curso. |
| forum_post_created | Publica un nuevo mensaje o respuesta en un foro. |
| forum_post_deleted | Elimina uno de sus propios mensajes en un foro. |
| forum_post_updated | Edita un mensaje que ya había publicado en un foro. |
| forum_subscription_created | Se suscribe a un foro completo (no solo a una discusión). |
| hvp_attempt_submitted | Envía un intento de actividad H5P (contenido interactivo). |
| hvp_module_viewed | Visualiza o abre un contenido interactivo H5P. |
| lti_module_viewed | Abre un recurso externo tipo LTI (integraciones de otras plataformas). |
| onlinetext_assessable_uploaded | Sube un texto en línea que será evaluado. |
| onlinetext_submission_created | Crea una entrega de texto online (como una redacción o ensayo). |
| page_module_viewed | Lee una página interna de contenido del curso. |
| quiz_attempt_reviewed | Revisa un intento de cuestionario (quiz) luego de enviarlo. |
| quiz_attempt_started | Inicia un intento de resolver un quiz. |
| quiz_attempt_submitted | Envía un quiz terminado para su calificación. |
| quiz_attempt_summary_viewed | Consulta el resumen de un intento de quiz. |
| quiz_attempt_viewed | Revisa detalles de un intento de quiz pasado. |
| quiz_module_viewed | Abre un módulo de cuestionario (quiz). |
| resource_module_viewed | Visualiza un recurso como un PDF, Word, etc. |
| submission_comment_created | Crea un comentario relacionado a una entrega hecha. |
| submission_status_viewed | Consulta el estado de su entrega en una tarea (por ejemplo: enviado, pendiente). |
| url_module_viewed | Abre un enlace externo sugerido en el curso. |


---

### Estructura de la tabla de logs (`moodle_logs.parquet` / MySQL 2024-2025)

| Campo | Descripción |
|:------|:------------|
| id | ID del evento en la base de datos |
| eventname | Nombre del evento (ej: course_viewed, quiz_attempt_started) |
| component | Componente de Moodle (ej: mod_forum, mod_quiz) |
| action | Acción realizada (ej: view, create, submit) |
| target | Objeto del evento (ej: course, resource, page) |
| objecttable | Tabla relacionada del objeto (opcional) |
| objectid | ID del objeto relacionado |
| crud | Tipo de acción CRUD (Create, Read, Update, Delete) |
| edulevel | Nivel educativo del evento (teaching, participation, other) |
| contextid | Contexto de ejecución del evento |
| contextlevel | Nivel del contexto (ej: curso, módulo) |
| contextinstanceid | ID específico del contexto |
| userid | ID del estudiante que realiza la acción |
| courseid | ID del curso donde ocurre el evento |
| relateduserid | ID de otro usuario relacionado (opcional) |
| anonymous | Indica si el evento fue anónimo |
| other | Información adicional del evento (JSON) |
| timecreated | Timestamp de creación del evento |
| origin | Origen de la acción (web, móvil, API) |
| ip | Dirección IP desde donde ocurrió el evento |
| realuserid | ID real del usuario (si fue actuando en nombre de otro) |

---

### Notas:
- El procesamiento de los eventos se limita a aquellos **relevantes para la interacción académica**.
- Se descartaron eventos técnicos o administrativos no pertinentes para el análisis de comportamiento de aprendizaje.



## 5. Docentes

### Descripción general
La fuente de datos de docentes contiene información sobre el personal educativo correspondiente a los periodos **2024-1 hasta 2025-1**.

---

### Variables disponibles

| Variable | Descripción |
|:---------|:------------|
| nombre | Nombre completo del docente. |
| edad | Edad del docente al momento del registro. |
| años_experiencia_ficc | Años de experiencia trabajando en la institución educativa FICC. |
| años_experiencia_total | Total de años de experiencia laboral en el área educativa. |
| nivel_educativo | Nivel educativo alcanzado (ej: pregrado, especialización, maestría). |
| rol_adicional | Función adicional desempeñada en la institución (ej: Coordinador, Director de grupo, etc.). |

---

### Notas:

- El dataset permite posteriormente analizar relaciones entre experiencia docente, nivel educativo y desempeño estudiantil.
- Se utilizará principalmente para enriquecer el análisis de contexto académico y variables de entorno educativo.



## 6. Carga Académica

### Descripción general
El dataset de **Carga Académica** contiene la relación entre los docentes, las asignaturas asignadas y la intensidad horaria correspondiente.

Los datos disponibles abarcan los periodos **2024-1 hasta 2025-1**.

---

### Información contenida

| Variable | Descripción |
|:---------|:------------|
| asignatura | Nombre de la asignatura asignada al docente. |
| intensidad_horaria | Número de horas semanales o periodo dedicadas a la asignatura. |
| docente_asignado | Nombre completo del docente encargado de la asignatura. |


